In [1]:
import pandas as pd
import numpy as np
import deepchem as dc
import rdkit 

/home/jykim/anaconda3/envs/automl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
/home/jykim/anaconda3/envs/automl/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/jykim/anaconda3/envs/automl/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
cellLine = pd.read_csv("/home/jykim/Project_poster/data_o/cell_info_filtered_662cell.csv")
response = pd.read_csv("/home/jykim/Project_poster/data_o/final_data_last.csv")
exp = pd.read_csv("/home/jykim/Project_poster/data_o/exp_filtered.csv")
mut = pd.read_csv("/home/jykim/Project_poster/data_o/mut_filtered.csv")
response.columns

Index(['COSMIC.ID', 'pubchem', 'response', 'SMILES', 'mfp1', 'mfp2', 'mfp3',
       'mfp4', 'mfp5', 'mfp6',
       ...
       'pc1', 'pc2', 'pc3', 'pc4', 'pc5', 'pc6', 'pc7', 'pc8', 'pc9', 'pc10'],
      dtype='object', length=1038)

In [3]:
import pubchempy as pcp

cid2smiles = dict()
for pid in response["pubchem"].unique().tolist():
    cid2smiles[pid] = pcp.Compound.from_cid(pid).isomeric_smiles

print(len(set(cid2smiles.values())))

307


In [4]:
response2 = response[['COSMIC.ID', 'pubchem', 'response', 'SMILES']]
response2["SMILES2"] = response2["pubchem"].map(cid2smiles)

/tmp/ipykernel_54895/831545455.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  response2["SMILES2"] = response2["pubchem"].map(cid2smiles)


In [5]:
import random 
res_ = response2[response2['response']==1]
nres_ = response2[response2['response']==0]
print(res_.shape, nres_.shape)

res_idx = random.sample(range(len(res_)), int(len(res_) * 0.8)) 
nres_idx = random.sample(range(len(nres_)), int(len(nres_) * 0.8))

res_train = res_.iloc[res_idx,:]
nres_train = nres_.iloc[nres_idx,:]

res_test = res_.iloc[list(set([i for i in range(len(res_))]) - set(res_idx)), :]
nres_test = nres_.iloc[list(set([i for i in range(len(nres_))]) - set(nres_idx)), :]

train_smiles = response2.loc[sorted(list(set(res_train.index)|set(nres_train.index))), :]
test_smiles = response2.loc[sorted(list(set(res_test.index)|set(nres_test.index))), :]
print(train_smiles.shape)
print(test_smiles.shape)

(53476, 5) (110252, 5)
(130981, 5)
(32747, 5)


In [15]:
test_smiles

,COSMIC.ID,pubchem,response,SMILES,SMILES2
0,946355,176870,1,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...
2,909703,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...
6,908467,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...
20,908451,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...
34,946359,176870,0,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...,COCCOC1=C(C=C2C(=C1)C(=NC=N2)NC3=CC=CC(=C3)C#C...
...,...,...,...,...,...
163712,1240223,51000408,0,CS(=O)(=O)C1=CC=C(C=C1)C2=CN=C(C(=N2)C(=O)NC3=...,CS(=O)(=O)C1=CC=C(C=C1)C2=CN=C(C(=N2)C(=O)NC3=...
163718,724828,51000408,0,CS(=O)(=O)C1=CC=C(C=C1)C2=CN=C(C(=N2)C(=O)NC3=...,CS(=O)(=O)C1=CC=C(C=C1)C2=CN=C(C(=N2)C(=O)NC3=...
163720,906798,51000408,0,CS(=O)(=O)C1=CC=C(C=C1)C2=CN=C(C(=N2)C(=O)NC3=...,CS(=O)(=O)C1=CC=C(C=C1)C2=CN=C(C(=N2)C(=O)NC3=...
163724,1298539,51000408,0,CS(=O)(=O)C1=CC=C(C=C1)C2=CN=C(C(=N2)C(=O)NC3=...,CS(=O)(=O)C1=CC=C(C=C1)C2=CN=C(C(=N2)C(=O)NC3=...


In [16]:
train_smiles.to_csv("/home/jykim/Project_poster/data_split/random_train_response.csv", index=False)
test_smiles.to_csv("/home/jykim/Project_poster/data_split/random_test_response.csv", index=False)

In [17]:
len(set(train_smiles["SMILES2"])&set(test_smiles["SMILES2"]))

307

In [19]:
train_smiles['response'].value_counts()

0    88201
1    42780
Name: response, dtype: int64

In [20]:
test_smiles['response'].value_counts()

0    22051
1    10696
Name: response, dtype: int64